In [ ]:
!pip install scapy
!pip install tensorflow keras
!pip install flask



In [ ]:
!pip install scapy
from scapy.all import sniff
def packet_callback(packet):
    print(packet.show())

sniff(prn=packet_callback, count=10)

In [ ]:
import random
import pandas as pd


def simulate_packet():
    return {
        'src_ip': f'192.168.0.{random.randint(1, 255)}',
        'dest_ip': f'192.168.1.{random.randint(1, 255)}',
        'protocol': random.choice(['TCP', 'UDP', 'ICMP']),
        'length': random.randint(60, 1500)
    }
data = []
for _ in range(1000):
    data.append(simulate_packet())

df = pd.DataFrame(data)

df['protocol'] = df['protocol'].map({'TCP': 0, 'UDP': 1, 'ICMP': 2})


df.head()

In [ ]:
try:
    from scapy.all import sniff
    print("Scapy instalado com sucesso!")
except ImportError:
    print("Erro ao instalar Scapy.")
try:
    import tensorflow as tf
    print("TensorFlow instalado com sucesso!")
except ImportError:
    print("Erro ao instalar TensorFlow.")

In [ ]:
import random
def simulate_packet():
    return { 'src_ip': f'192.168.0.{random.randint(1, 255)}',
        'dest_ip': f'192.168.1.{random.randint(1, 255)}',
        'protocol': random.choice(['TCP', 'UDP', 'ICMP']),
        'length': random.randint(60, 1500)
    }
def packet_callback(packet):
    print(packet)
for _ in range(10):
    packet = simulate_packet()
    packet_callback(packet)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
print("Bibliotecas importadas com sucesso!")

In [ ]:
data = []
for _ in range(1000):
    data.append(simulate_packet())

df = pd.DataFrame(data)

df['protocol'] = df['protocol'].map({'TCP': 0, 'UDP': 1, 'ICMP': 2})
df.head()

In [31]:
X = df[['src_ip', 'dest_ip', 'protocol', 'length']]

X['src_ip'] = X['src_ip'].apply(lambda x: int(x.split('.')[3]))
X['dest_ip'] = X['dest_ip'].apply(lambda x: int(x.split('.')[3]))


scaler= StandardScaler()
X_scaled= scaler.fit_transform(X)


X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

print("Dados pré-processados com sucesso!")


Dados pré-processados com sucesso!


In [32]:
input_dim = X_train.shape[1]

input_layer = Input(shape=(input_dim,))
encoder = Dense(4, activation="relu")(input_layer)
encoder = Dense(2, activation="relu")(encoder)
decoder = Dense(4, activation='relu')(encoder)
decoder = Dense(input_dim, activation='sigmoid')(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

print("Modelo de Autoencoder criado com sucesso!")

Modelo de Autoencoder criado com sucesso!


In [ ]:
history = autoencoder.fit(X_train, X_train,
                          epochs=50,
                          batch_size=16,
                          validation_data=(X_test, X_test))
print("modelo treinado com sucesso")

In [23]:
predictions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - predictions, 2), axis=1)
threshold = np.percentile(mse, 95)
anomalies = mse > threshold
print(f"Numero de anomalias detectadas: {np.sum(anomalies)}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Numero de anomalias detectadas: 10


In [ ]:
!pip install plotly

In [34]:
import plotly.express as px
results = pd.DataFrame(X_test, columns=['src_ip', 'dest_ip', 'protocol', 'length'])
results['mse'] = mse
results['anomaly'] = anomalies
fig = px.scatter(results, x='src_ip', y='mse', color='anomaly', title="Detecção de Anomalias")
fig.show()